In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import geopandas as gpd

In [3]:
df = pd.read_csv('declarations-exterminations-punaises-de-lit.csv')

In [4]:
df['DATE_DECLARATION'] = pd.to_datetime(df['DATE_DECLARATION'])
df['DATE_DEBUTTRAIT'] = pd.to_datetime(df['DATE_DEBUTTRAIT'])
df['DATE_FINTRAIT'] = pd.to_datetime(df['DATE_FINTRAIT'])
df['DATE_INSP_VISPRE'] = pd.to_datetime(df['DATE_INSP_VISPRE'])

df['LEN_TRAIT'] = df['DATE_FINTRAIT'] - df['DATE_DEBUTTRAIT'] 
df['LEN_TRAIT'] = df['LEN_TRAIT'].dt.days
df['NUM_WEEKS_TRAIT'] = np.ceil(df['LEN_TRAIT']/7)

In [5]:
df_bb = df[df['NUM_WEEKS_TRAIT']<=54]

The date of the event is the minimum of date declaration, date inspection and date beginning of treatment, where the date that is kept is the one where at least 2 events occurred not so far apart. 

In [6]:
len(df_bb)

32783

In [7]:
condition = ((df_bb['DATE_DEBUTTRAIT'] - df_bb['DATE_DECLARATION']).dt.days > -365) & ((df_bb['DATE_DEBUTTRAIT'] - df_bb['DATE_INSP_VISPRE']).dt.days > -365)

In [8]:
condition2 = ((df_bb['DATE_DEBUTTRAIT'] - df_bb['DATE_DECLARATION']).dt.days < 365) & ((df_bb['DATE_DEBUTTRAIT'] - df_bb['DATE_INSP_VISPRE']).dt.days < 365)

In [9]:
df_bb = df_bb[condition2]

In [10]:
df_bb = df_bb[condition]

C:\Users\Mélisande\Anaconda3\envs\VRDI\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [11]:
len(df_bb)

32486

In [12]:
df_bb['DATE_EVENT'] = df_bb[['DATE_DECLARATION','DATE_INSP_VISPRE','DATE_DEBUTTRAIT']].min(axis = 1)

df_bb['YEAR_EVENT'] = df_bb['DATE_EVENT'].dt.year
df_bb['MONTH_EVENT'] = df_bb['DATE_EVENT'].dt.month
df_bb['WEEK_EVENT'] = df_bb['MONTH_EVENT']*(1 + df_bb['DATE_EVENT'].dt.day // 7)

In [13]:
df_bb.columns

Index(['NO_DECLARATION', 'DATE_DECLARATION', 'DATE_INSP_VISPRE',
       'NBR_EXTERMIN', 'DATE_DEBUTTRAIT', 'DATE_FINTRAIT', 'No_QR', 'NOM_QR',
       'NOM_ARROND', 'COORD_X', 'COORD_Y', 'LONGITUDE', 'LATITUDE',
       'LEN_TRAIT', 'NUM_WEEKS_TRAIT', 'DATE_EVENT', 'YEAR_EVENT',
       'MONTH_EVENT', 'WEEK_EVENT'],
      dtype='object')

In [14]:
df_bb_ = df_bb[[ 'NO_DECLARATION', 'No_QR', 'COORD_X', 'COORD_Y', 'NBR_EXTERMIN', 'DATE_EVENT', 'YEAR_EVENT',
       'MONTH_EVENT', 'WEEK_EVENT', 'LEN_TRAIT', 'NUM_WEEKS_TRAIT', 'NOM_QR',
       'NOM_ARROND', 'DATE_DECLARATION', 'DATE_INSP_VISPRE', 'DATE_DEBUTTRAIT']]

In [15]:
df_bb_['YEAR_EVENT'].min()

2011

In [16]:
df_bb_['DATE_EVENT'].isna().value_counts()

False    32486
Name: DATE_EVENT, dtype: int64

In [17]:
from shapely.geometry import Point
import shapely

In [18]:
shapefiles = gpd.read_file('./quartierreferencehabitation/QuartierReferenceHabitation.shp', encoding = 'utf-8')

In [19]:
shapefiles = shapefiles[shapefiles['nom_mun'] == 'Montréal']

In [20]:
shapefiles['CENTROID'] = shapefiles.centroid

In [21]:
shapefiles['CENTROID_X'] =shapefiles.centroid.x
shapefiles['CENTROID_Y'] =shapefiles.centroid.y


In [22]:
len(shapefiles['CENTROID_X'].unique())

77

In [23]:
df_ = df_bb_.merge(shapefiles[['no_qr', 'CENTROID_X', 'CENTROID_Y']], left_on = 'No_QR', right_on = 'no_qr')

In [24]:
df_.to_csv('bbugs_clean.csv', index=False)

In [25]:
df_.columns

Index(['NO_DECLARATION', 'No_QR', 'COORD_X', 'COORD_Y', 'NBR_EXTERMIN',
       'DATE_EVENT', 'YEAR_EVENT', 'MONTH_EVENT', 'WEEK_EVENT', 'LEN_TRAIT',
       'NUM_WEEKS_TRAIT', 'NOM_QR', 'NOM_ARROND', 'DATE_DECLARATION',
       'DATE_INSP_VISPRE', 'DATE_DEBUTTRAIT', 'no_qr', 'CENTROID_X',
       'CENTROID_Y'],
      dtype='object')